# Chapter 05. 트리 알고리즘

## 05-2. 교차 검증과 그리드 서치

### Validation set
- test set 으로 자꾸 성능을 확인하게 되면 과대적합이 발생할 수 있다.
- 일반화 성능을 올바르게 측정하기 위해 검증 세트 도입.
- 모델 성능 평가 과정 <br>
  훈련 세트에서 훈련하고, 검증 세트에서 모델을 평가한다. <br>
  여러 매개변수의 경우 중 가장 성능이 좋은 모델을 골라 훈련 세트에서 다시 훈련한다.

#### 데이터 불러오기

In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
data = wine [['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# validation set 설정 (train set에서 다시 쪼개기)
sub_input, val_input, sub_target, var_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


#### 의사결정나무 분류

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
dt = DecisionTreeClassifier(random_state=42)

In [8]:
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [9]:
print(dt.score(sub_input, sub_target))

0.9971133028626413


In [10]:
print(dt.score(val_input, var_target))

0.864423076923077


### Cross Validation
- 교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.
- 교차 검증 과정 <br>
  훈련 세트에서 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. <br>
  반복하면서 나온 검증 점수들을 평균하여 최종 검증 점수를 얻는다.
<br><br>
- 교차 검증 함수 : cross_validate() => default : 5-Fold Cross Validation <br>
     fit_time : 모델 훈련 시간 <br>
     score_time : 검증 시간 <br>
     test_score : 교차 검증 점수 ( 최종 교차 검증 점수는 평균해야함 : np.mean(scores['test_score']) )
<br><br>
- 교차 검증 종류 : K-Fold Cross Validation // StratifiedKFold Cross Validation
- K-Fold Cross Validation <br>
  : 회귀 모델인 경우 사용
- StratifiedKFold Cross Validation <br>
  : 분류 모델인 경우 사용 <br>
  : 불균형 분포도를 가진 레이블 집합을 위한 K-Fold 방식

#### 5-Fold Cross Validation

In [11]:
from sklearn.model_selection import cross_validate

In [12]:
scores = cross_validate(dt, train_input, train_target)

In [13]:
print(scores)

{'fit_time': array([0.00998878, 0.00697947, 0.00897598, 0.00598383, 0.00698137]), 'score_time': array([0.00098205, 0.        , 0.00099754, 0.00099754, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [14]:
import numpy as np
print(np.mean(scores['test_score']))   # 교차 검증의 최종 점수

0.855300214703487


#### StratifiedKFold
- cross_validate()는 데이터를 섞지 않는다. <br>
  train_target_split()을 사용하지 않고 바로 교차 검증을 들어가는 경우 반드시 분할기(splitter)을 지정해야 한다.

In [15]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [16]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)   # n_splits=10 : 10-Fold Cross Validation
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


### HyperParameter Tuning
- 사용자가 지정하는 파라미터
- 여러 경우의 수를 이용하여 최적의 값을 찾아 모델의 성능을 올려야 한다. <br>
  이때 그리드 서치(Grid Search)를 이용하여 매개변수의 여러 경우의 수를 따져본다.
<br><br>
- Grid Search <br>
  : best_estimator_ 속성에 검증 점수가 가장 높은 모델이 저장되어 있다. <br>
  : best_params_ 속성에 검증 점수가 가장 높은 최적의 매개변수 조합이 저장되어 있다. <br>
  : model.cv_results_['mean_test_score'] 에 각 매개변수 조합으로 수행한 교차 검증의 평균 점수가 저장되어 있다. <br>
  : n_jobs 컴퓨터의 CPU 코어 수를 지정하는 매개변수 (코어 수에 비례해 속도가 빨라짐. default=1)

#### Grid Search

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)   # n_jobs=-1 : 시스템에 있는 모든 코어 사용

In [20]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [21]:
dt = gs.best_estimator_

In [22]:
print(dt.score(train_input, train_target))

0.9615162593804117


In [23]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [24]:
print(gs.cv_results_['mean_test_score'])   # 가장 큰 score가 최종 교차 검증 모델 (아래에서 넘파이 함수로 쉽게 구하기)

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [25]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


#### 더 복잡한 매개변수 조합 탐색

In [26]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [28]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### Random Search
- 랜덤 서치는 매개변수를 샘플링할 수 있는 확률 분포를 객체에 전달한다. <br>
  즉 매개변수 샘플링을 고르게 할 수 있다. (임의로 간격을 지정하지 않아도 된다.)

In [30]:
from scipy.stats import uniform, randint

#### randint, uniform 예시

In [31]:
rgen = randint(0,10)
rgen.rvs(10)

array([7, 3, 9, 4, 1, 6, 0, 9, 5, 2])

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92, 110,  96,  98, 105,  96,  97,  88, 118, 100], dtype=int64))

In [33]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.02361803, 0.91883422, 0.76530481, 0.52323127, 0.17039236,
       0.08657267, 0.72483092, 0.30700134, 0.81383747, 0.29706654])

#### 랜덤 서치와 그리드 서치를 이용한 매개변수 조합 탐색

In [34]:
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
# n_iter=100 : 매개변수 범위 안에서 총 100번 샘플링

In [37]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75C5C0A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75BC81F0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75C5C730>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75BC8880>},
                   random_state=42)

In [38]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [40]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8928227823744468


#### 번외
- DecisionTreeClassifier에 매개변수 splitter='random'를 추가하면 성능이 어떻게 될까?

In [41]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params, n_iter=100, n_jobs=-1,
                        random_state=42)   

In [42]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75C5C0A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75BC81F0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75C5C730>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001EA75BC8880>},
                   random_state=42)

In [43]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8043101789493938


#### 성능이 약 0.09 떨어진다.